In [1]:
import sys
print(sys.executable)

/home/rs/PycharmProjects/snake-multi/.venv/bin/python3.10


In [2]:
import os
os.chdir("../..")
os.getcwd()

'/home/rs/PycharmProjects/snake-multi'

In [3]:
import random
import logging
import sys
from collections import deque
from functools import partial
from math import floor, ceil
from pathlib import Path

import tensorflow as tf
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from src.simulation.simulation import Simulation, Vector2
from src.simulation.board_generator import make_simple_board

from src.model.model_utils import get_model_moves, get_model_scores, get_rotated_state, ReplayBuffer

tf.config.experimental.set_memory_growth(tf.config.list_physical_devices("GPU")[0], True)

2025-09-12 18:01:50.638698: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1757692910.650359 2767071 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1757692910.654023 2767071 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1757692910.664054 2767071 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757692910.664070 2767071 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1757692910.664072 2767071 computation_placer.cc:177] computation placer alr

In [4]:
view_range = 15

replay_buffer_size = 50_000
replay_buffer_threshold_multiplier = 1.3

discount_factor = 0.9

batch_size = 32
loss_fn = tf.keras.losses.mse
optimizer = tf.keras.optimizers.Nadam(learning_rate=0.001)

n_episodes = 1_000

best_model_window = 7

I0000 00:00:1757692912.421250 2767071 gpu_device.cc:2019] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1652 MB memory:  -> device: 0, name: NVIDIA GeForce GTX 1060 3GB, pci bus id: 0000:01:00.0, compute capability: 6.1


In [5]:
def calculate_score(ate_food: bool, died: bool, n_snakes_killed: int):
    if died:
        return -10

    reward = 0
    
    if ate_food:
        reward += 5

    reward += n_snakes_killed * 10

    return reward

In [6]:
Dense = partial(tf.keras.layers.Dense, activation="relu", kernel_initializer="he_normal")

online_model = tf.keras.Sequential([
    tf.keras.layers.Input((5 * 8,)),
    tf.keras.layers.Rescaling(2 / view_range, -1),
    Dense(64),
    Dense(64),
    Dense(64),
    tf.keras.layers.Dense(1)
])

target_model = tf.keras.models.clone_model(online_model)
target_model.set_weights(online_model.get_weights())

view_type = "diagonal"

In [7]:
def fix_float(val: float) -> str:
    return str(round(float(val), 8)).replace('.', '_')

model_name = f"r{view_range}_{view_type}_rb_{fix_float(replay_buffer_threshold_multiplier)}_e_{n_episodes}_lr_{fix_float(optimizer.learning_rate)}"

model_name += "timeout_scaling_battle"

In [8]:
for i in range(100):
    file_path = Path(f"logs_{model_name}_v{i}.log")

    if not file_path.is_file():
        break
else:
    raise RuntimeError("If it's not a bug, train less models")

logging.basicConfig(level=logging.DEBUG)

log_formatter = logging.Formatter("%(message)s")

std_handler = logging.StreamHandler(sys.stdout)
std_handler.setLevel(logging.INFO)
std_handler.setFormatter(log_formatter)

file_handler = logging.FileHandler(file_path)
file_handler.setLevel(logging.INFO)
file_handler.setFormatter(log_formatter)

logger = logging.getLogger()
logger.handlers.clear()
logger.addHandler(std_handler)
logger.addHandler(file_handler)

In [9]:
online_model.summary(print_fn=logger.info)
online_model.summary()

Model: "sequential"
┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴─────────────────────

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ rescaling (Rescaling)           │ (None, 40)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         2,624 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 64)             │         4,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 1)              │            65 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,009 (43.00 KB)

 Trainable params: 11,009 (43.00 KB)

 Non-trainable params: 0 (0.00 B)

In [10]:
# Make sure model takes correct input
simulation = Simulation(calculate_score, False)
simulation.reset(make_simple_board(np.array([view_range * 2 + 1, view_range * 2 + 1])), 2, 1, 1)

online_model.predict(np.array([simulation.get_snake_view(0, view_type, view_range)]))

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 147ms/step


I0000 00:00:1757692912.974524 2767139 service.cc:152] XLA service 0x7f1ed00044e0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1757692912.974551 2767139 service.cc:160]   StreamExecutor device (0): NVIDIA GeForce GTX 1060 3GB, Compute Capability 6.1
2025-09-12 18:01:52.979475: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1757692912.993122 2767139 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1757692913.075598 2767139 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


array([[0.43501905]], dtype=float32)

In [11]:
def epsilon_greedy(snakes_states: list[np.ndarray], snakes_possible_moves: list[list[Vector2]], snakes_alive: list[bool], epsilon: float=0.):
    n_snakes = len(snakes_states)

    process_mask = [True for _ in range(n_snakes)]
    
    for i, alive in enumerate(snakes_alive):
        if not alive or np.random.rand() < epsilon:
            process_mask[i] = False

    moves = get_model_moves(online_model, snakes_states, snakes_possible_moves, process_mask, softmax=True)

    for i, move in enumerate(moves):
        if move == (0, 0):
            moves[i] = random.choice(simulation.get_legal_moves(i))

    return moves

In [12]:
#priority_replay_buffer = deque(maxlen=replay_buffer_size // 2)
#replay_buffer = deque(maxlen=replay_buffer_size // 2)

replay_buffer = ReplayBuffer(
    priority_size=replay_buffer_size // 2,
    normal_size=replay_buffer_size // 2,
    threshold_multiplier=replay_buffer_threshold_multiplier
)

In [13]:
def play_one_step(simulation: Simulation, states_before: list[np.ndarray], possible_moves_before: list[Vector2], snakes_alive_before: list[bool], epsilon: float):
    # Save states
    moves = epsilon_greedy(states_before, possible_moves_before, snakes_alive_before, epsilon)

    scores, running = simulation.next(moves)

    states_after = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves_after = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive_after = simulation.snakes_alive

    for i in range(len(states_before)):
        if snakes_alive_before[i]:
            replay_buffer.append((states_before[i], moves[i], scores[i], states_after[i], possible_moves_after[i], snakes_alive_after[i]))

    return states_after, possible_moves_after, snakes_alive_after, scores, running

In [14]:
def sample_experiences(batch_size: int):
    idxs = np.random.randint(len(replay_buffer), size=batch_size)
    
    batch = [replay_buffer[idx] for idx in idxs]

    return [
        [experience[field_idx] for experience in batch]
        for field_idx in range(6)
    ]

In [15]:
losses = []


def training_step(buffer_split: float):
    #experiences = sample_experiences(batch_size)
    experiences = replay_buffer.sample(split=buffer_split, batch_size=batch_size)
    states, moves, scores, states_after, possible_moves_after, snakes_alive_after = experiences

    best_next_moves = [[move] for move in get_model_moves(online_model, states_after, possible_moves_after, snakes_alive_after)]
    next_q_values = get_model_scores(target_model, states_after, best_next_moves, snakes_alive_after)
    
    #next_q_values = get_model_scores(target_model, states_after, possible_moves_after, snakes_alive_after)
    runs = 1.0 - np.array(snakes_alive_after)
    
    target_q_values = scores + runs * discount_factor * next_q_values

    X = tf.constant([get_rotated_state(state, move) for state, move in zip(states, moves)])
    
    with tf.GradientTape() as tape:
        predicted_q_values = online_model(X)
        individual_losses = loss_fn(target_q_values, predicted_q_values)
        loss = tf.reduce_mean(individual_losses)

    replay_buffer.update_loss(individual_losses, loss)

    losses.append(loss.numpy())
    grads = tape.gradient(loss, online_model.trainable_variables)
    optimizer.apply_gradients(zip(grads, online_model.trainable_variables))

In [16]:
simulation = Simulation(calculate_score, True)
rewards = []
steps = []
replay_buffer.clear()

best_model_rewards = float("-inf")

for episode in range(n_episodes):
    simulation.reset(make_simple_board(np.array([15, 15])), snake_count=2, food_count=1, tail_len=2)

    rewards.append(0)

    states = [simulation.get_snake_view(i, view_type, view_range) for i in range(simulation.n_snakes)]
    possible_moves = [simulation.get_legal_moves(i) for i in range(simulation.n_snakes)]
    snakes_alive = simulation.snakes_alive
    
    epsilon = max(1 - (episode / (n_episodes * 0.8)), 0.005)
    priority_buffer_split = min(episode / (n_episodes * 0.8) * 0.2, 0.2)

    step = 0
    
    while True:
        states, possible_moves, snakes_alive, scores, running = play_one_step(simulation, states, possible_moves, snakes_alive, epsilon)

        rewards[-1] += sum(scores)

        if not running:
            break

        step += 1

    steps.append(step)

    logging.info(f"Episode {episode} played")
    logging.info(f"\tSteps: {step}")
    logging.info(f"\tRewards: {rewards[-1]}")

    if episode > 200:
        curr_model_rewards = sum(rewards[-best_model_window:])
        if curr_model_rewards > best_model_rewards:
            avg_rewards = curr_model_rewards / best_model_window
            best_model_rewards = curr_model_rewards
            logging.info(f"\tSnaphshot saved (avg. {avg_rewards})")
            
            if model_name:
                online_model.save(f"models/{model_name}_{round(avg_rewards)}_snapshot.keras")

        training_step(priority_buffer_split)
        
        logger.info(f"\tPriority: {len(replay_buffer.priority_buffer)}")
        
        if episode % 100 == 0:
            target_model.set_weights(online_model.get_weights())
            logging.info("\tTarget model updated")

Episode 0 played
	Steps: 10
	Rewards: -10
Episode 1 played
	Steps: 28
	Rewards: -10
Episode 2 played
	Steps: 21
	Rewards: -10
Episode 3 played
	Steps: 4
	Rewards: -10
Episode 4 played
	Steps: 17
	Rewards: -10
Episode 5 played
	Steps: 13
	Rewards: 0
Episode 6 played
	Steps: 12
	Rewards: -10
Episode 7 played
	Steps: 3
	Rewards: 0
Episode 8 played
	Steps: 3
	Rewards: -10
Episode 9 played
	Steps: 33
	Rewards: 0
Episode 10 played
	Steps: 20
	Rewards: 0
Episode 11 played
	Steps: 22
	Rewards: -10
Episode 12 played
	Steps: 11
	Rewards: -10
Episode 13 played
	Steps: 16
	Rewards: -10
Episode 14 played
	Steps: 21
	Rewards: -10
Episode 15 played
	Steps: 5
	Rewards: 0
Episode 16 played
	Steps: 24
	Rewards: -10
Episode 17 played
	Steps: 16
	Rewards: -10
Episode 18 played
	Steps: 4
	Rewards: -10
Episode 19 played
	Steps: 13
	Rewards: 0
Episode 20 played
	Steps: 22
	Rewards: -10
Episode 21 played
	Steps: 8
	Rewards: -10
Episode 22 played
	Steps: 3
	Rewards: -10
Episode 23 played
	Steps: 11
	Rewards: -